In [1]:
!pip install kaggle

In [2]:
!pip install tensorflow

In [3]:
import tensorflow as tf
print(tf.__version__)


2.18.0


Importing dependencies

In [ ]:
#to access config files, read those files
#import os
#import json

#from zipfile import ZipFile #extract zipfiles
#import pandas as pd
#from tensorflow import keras
#from sklearn.model_selection import train_test_split
#from keras import Sequential #building linear Neural Net
#from keras import Dense, Embedding, LSTM #the three layers
#from keras.preprocessing.text import Tokenizer #to tokenize the data => map data to certain integers or vectors
#from keras.preprocessing.sequence import pad_sequences #to ensure all input data of same shape

In [4]:
import os
import json
import tensorflow as tf
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
#from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection - Kaggle API

In [6]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [7]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [8]:
#setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [9]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


Loading the data

In [11]:
!ls

 drive		     imdb-dataset-of-50k-movie-reviews.zip   sample_data
'IMDB Dataset.csv'   kaggle.json


In [12]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [13]:
!ls

 drive		     imdb-dataset-of-50k-movie-reviews.zip   sample_data
'IMDB Dataset.csv'   kaggle.json


In [14]:
data = pd.read_csv("IMDB Dataset.csv")

In [15]:
data.shape

(50000, 2)

In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [17]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [18]:
data["sentiment"].value_counts() #checking if the data is balanced (equal number of instances for both classes)

,count
sentiment,
positive,25000
negative,25000


In [19]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)

<ipython-input-19-e845730812f2>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)


In [20]:
#to check encoding
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [21]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [22]:
#split dataset
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42) #20% of test data, 80% training data #random_state: decides randomization

In [23]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Preprocessing

In [24]:
#Tokenizing the text data
tokenizer = Tokenizer(num_words=5000) #convert words or phrases to an integer; consider most common 5000 words from the entire data
tokenizer.fit_on_texts(train_data["review"]) #fit the review texts to the training data
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen = 200) #converting 5000 words into integers and sequencing those integers; pad_seq makes sure that all train_data has the same length
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen = 200)

In [25]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [26]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [27]:
Y_train = train_data["sentiment"] #sentiment column is the column in "data"
Y_test = test_data["sentiment"]

In [28]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


LSTM - Long Short Term Memory
- type of a RNN
- useful for sequence dataset
- time series (say Apr 1: weather is 80, Apr 2: weather is 85)

In [29]:
# Building model
'''
model = Sequential()
# Representing data in vector form
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))  #Layer 1: I/P 5000 as there were 5000 reviews, O/P = 128 (each word represented as 128 dimension vector space); I/P length = maxlen of X_train/X_test
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) #Layer 2: 128 neurons in LSTM, dropout = 20% of the data - to avoid overfit (regularization parameter: dropout, recurrent_dropout)
model.add(Dense(1, activation="sigmoid")) #Layer 3: O/P layer with 1 neuron as O/P -> giving 0 or 1; Sigmoid: binary classification

# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

'''
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input

# Define input explicitly
input_layer = Input(shape=(200,))  # Input is a sequence of 200 integers

# Embedding Layer
embedding = Embedding(input_dim=5000, output_dim=128)(input_layer)

# LSTM Layer
lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(embedding)

# Output Layer
output = Dense(1, activation="sigmoid")(lstm)

# Build model
model = Model(inputs=input_layer, outputs=output)

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [30]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

### Model Training ###

In [31]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 213s 410ms/step - accuracy: 0.7328 - loss: 0.5264 - val_accuracy: 0.8575 - val_loss: 0.3408
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 248s 393ms/step - accuracy: 0.8403 - loss: 0.3734 - val_accuracy: 0.8643 - val_loss: 0.3263
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 195s 391ms/step - accuracy: 0.8849 - loss: 0.2868 - val_accuracy: 0.8608 - val_loss: 0.3428
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 392ms/step - accuracy: 0.8785 - loss: 0.2955 - val_accuracy: 0.8711 - val_loss: 0.3165
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 200s 389ms/step - accuracy: 0.9113 - loss: 0.2276 - val_accuracy: 0.8771 - val_loss: 0.3007


### Model Evaluation ###

In [32]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 109ms/step - accuracy: 0.8855 - loss: 0.2850
Test Loss: 0.2881750762462616
Test Accuracy: 0.8842999935150146


### Building a Predictive System ###

In [33]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review]) #review is tokenized
  padded_sequence = pad_sequences(sequence, maxlen=200) #input data has the same length as the data trained on
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

### Testing Model Prediction for New Reviews ###

In [34]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step
The sentiment of the review is: positive


In [35]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
The sentiment of the review is: negative


In [40]:
new_review = "This movie was thrilling"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
The sentiment of the review is: negative


In [36]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
The sentiment of the review is: negative


# Debug

In [ ]:
print(type(X_train))  # Should be <class 'numpy.ndarray'>
print(X_train.shape)  # Expected: (num_samples, 200)


<class 'numpy.ndarray'>
(40000, 200)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=(200, 128)))  # Explicit input_shape
model.add(Dense(1, activation="sigmoid"))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_16 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input

# Define input explicitly
input_layer = Input(shape=(200,))  # Input is a sequence of 200 integers

# Embedding Layer
embedding = Embedding(input_dim=5000, output_dim=128)(input_layer)

# LSTM Layer
lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(embedding)

# Output Layer
output = Dense(1, activation="sigmoid")(lstm)

# Build model
model = Model(inputs=input_layer, outputs=output)

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_17 (Embedding)             │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)